In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from scipy.special import comb

We choose 20 students, and we ask each of them to flip a coin 10 times and record the number of heads they got. The results are:

In [2]:
data = pd.read_excel('Lecture_2_2_coin_flip.xlsx')
data.head()

,flips,heads
0,10,4
1,10,4
2,10,3
3,10,1
4,10,6


Model this process using the binomial distribution. If n is the number of flips, k is the number of heads, and p is the probability of getting heads on a single flip, then the probability of getting k heads out of n flips is:

${n \choose k} p^k (1-p)^{(n-k)}$

There is only one parameter in this model: $p$, the probability of getting heads on a single flip.

In [3]:
def binom(params):
    p = params
    prob_k_heads = comb(data['flips'], data['heads']) * p**data['heads'] * (1 -p)**(data['flips'] - data['heads'])
    return prob_k_heads

If we know the value of $p$, we can find the probability of the given number of heads for each student. For example, if we set $p = 0.5$, 

In [4]:
data['prob_that_many_heads'] = binom(0.5)
data.head()

,flips,heads,prob_that_many_heads
0,10,4,0.205078
1,10,4,0.205078
2,10,3,0.117188
3,10,1,0.009766
4,10,6,0.205078


In [5]:
data.prob_that_many_heads.prod()

2.1018475030026418e-19

The probability of getting this exact configuration for all 20 students is the product of all values of prob_that_many_heads. Since this total probability is so small, we will work with its logarithm.

In [6]:
def MLE_binom(params):
    return -np.sum(np.log(binom(params)))

Let the computer try different values of $p$ and report the one with the highest probability for all 20 students (this will be the one with the lowest value of the sum of log likelihoods). Make a starting guess of $p = 0.5$, since this is what the probability would be for a fair coin.

In [7]:
starting_val = [0.5]
bounds = [(0, 1)]
mle_model = minimize(MLE_binom, starting_val, method='Nelder-mead', bounds=bounds)
mle_model

 final_simplex: (array([[0.35      ],
       [0.35009766]]), array([33.86619174, 33.86619593]))
           fun: 33.86619174052527
       message: 'Optimization terminated successfully.'
          nfev: 26
           nit: 13
        status: 0
       success: True
             x: array([0.35])

This value of p is lower than 0.5. The coin may not be fair.

## Discrete heterogeneity

Now consider that perhaps one coin was used in some of the experiments, and a second coin was used on the other experiments. How can this be modeled? 

There will be 2 binomial distributions, each with its own value of $p$. There must also be a probability that the measurement came from the first distribution and a probability that the measurement came from the second distribution - in other words, the probability that an experiment used the first coin or the second coin.

$P(k) = f {n \choose k} p_1^k (1-p_1)^{(n-k)} + (1-f) {n \choose k} p_2^k (1-p_2)^{(n-k)}$

Modify the functions used to solve this problem.

In [8]:
def binom_2(params):
    p1, p2, f= params
    prob_k_heads_1 = comb(data['flips'], data['heads']) * p1**data['heads'] * (1 -p1)**(data['flips'] - data['heads'])
    prob_k_heads_2 = comb(data['flips'], data['heads']) * p2**data['heads'] * (1 -p2)**(data['flips'] - data['heads'])
    prob_k_heads = f * prob_k_heads_1 + (1 - f) * prob_k_heads_2
    return prob_k_heads

In [9]:
def MLE_binom_2(params):
    return -np.sum(np.log(binom_2(params)))

Let the computer try different values of $p_1$, $p_2$, and $f$ and report the set with the highest probability for all 20 students (this will be the one with the lowest value of the sum of log likelihoods). Make a starting guess of $p_1 = 0.25$, $p_2 = 0.75$, and $f = 0.75$. Although both $p_1$ and $p_2$ could be set to the same value, it's not wise to do so because every time the solution is found, $p_1$ and $p_2$ could reverse values. $p_1$, $p_2$, and $f$ are probabilities and are bounded by 0 and 1. 

In [10]:
starting_val = [0.25, 0.75, 0.75]
bounds = [(0, 1), (0, 1), (0, 1)]
mle_model = minimize(MLE_binom_2, starting_val, method='Nelder-mead', bounds=bounds)
mle_model

 final_simplex: (array([[0.34999977, 0.34999644, 0.77855854],
       [0.35000254, 0.34999753, 0.77865111],
       [0.35000265, 0.34999563, 0.77848749],
       [0.349997  , 0.35000545, 0.77857371]]), array([33.86619174, 33.86619174, 33.86619174, 33.86619174]))
           fun: 33.86619174116445
       message: 'Optimization terminated successfully.'
          nfev: 103
           nit: 57
        status: 0
       success: True
             x: array([0.34999977, 0.34999644, 0.77855854])

The same value is found for $p_1$ and $p_2$, meaning that it is likely that only one coin was used for all experiments.

## Continuous heterogeneity

In this model, each experiment has its own coin with its own value of $p$, the probability of getting heads. It would be possible to model this by using 20 different parameters $p_1$, $p_2$, ..., $p_{20}$, but this model is so complicated that it would be nearly impossible to find a good solution. Instead, model the distribution of values of $p$ with a beta distribution. The final model is the beta-binomial distribution, which has the 2 parameters $\alpha$ and $\beta$:

$P(k) = \frac{\Gamma(n+1)}{\Gamma(k+1)\Gamma(n-k+1)}\frac{\Gamma(k+\alpha)\Gamma(n-k+\beta)}{\Gamma(n+\alpha+\beta)}\frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)}$

$\alpha$ and $\beta$ are bounded below by 0.

In [11]:
from scipy.special import gamma
def binom_beta(params):
    alpha, beta = params

    term1 = gamma(data['flips'] + 1)/(gamma(data['heads'] + 1) * gamma(data['flips'] - data['heads'] + 1))
    term2 = gamma(data['heads'] + alpha) * gamma(data['flips'] - data['heads'] + beta)/gamma(data['flips'] + alpha + beta)
    term3 = gamma(alpha + beta)/(gamma(alpha) * gamma(beta))

    prob = term1 * term2 * term3

    return prob

In [12]:
def MLE_binom_beta(params):
    return -np.sum(np.log(binom_beta(params)))

In [13]:
starting_vals = [1, 1]
bounds = [(0, 500), (0, 500)]
mle_model_binom_beta = minimize(MLE_binom_beta, starting_vals, method='Nelder-mead', bounds=bounds)
mle_model_binom_beta

C:\Users\rston\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\rston\AppData\Local\Temp/ipykernel_4912/4083245400.py:7: RuntimeWarning: overflow encountered in double_scalars
  term3 = gamma(alpha + beta)/(gamma(alpha) * gamma(beta))
C:\Users\rston\AppData\Local\Temp/ipykernel_4912/4083245400.py:7: RuntimeWarning: invalid value encountered in double_scalars
  term3 = gamma(alpha + beta)/(gamma(alpha) * gamma(beta))


 final_simplex: (array([[ 62.70634766, 111.38781738],
       [ 62.70636526, 111.38785401],
       [ 62.70637788, 111.3878718 ]]), array([-0., -0., -0.]))
           fun: -0.0
       message: 'Optimization terminated successfully.'
          nfev: 124
           nit: 47
        status: 0
       success: True
             x: array([ 62.70634766, 111.38781738])

In [16]:
mle_model_binom_beta.x[0]/(mle_model_binom_beta.x[0] + mle_model_binom_beta.x[1])

0.36018638328389524

The fit of the solution is good, but the parameters don't have a simple interpretation.

The mean is $\alpha/(\alpha + \beta)$ which is approximately 0.36. This verifies that there is only one group, and it has a probability of getting heads equal to 0.35 (approximately).

## Observed heterogeneity

Assume a sensitive instrument has been used to measure the weight of each side of the coins. If the tails side weighs more than the heads side, the number is positive and vice versa. The measured values, in milligrams, are shown below. 

It is possible that when a coin is heavier on the tails side, it is more likely to land heads-up. Do the data support this hypothesis?

In [18]:
data = pd.read_csv('Lecture_2_2_coin_flip.csv', index_col=None)

In [19]:
data

,flips,heads,mg_diff
0,10,4,-0.903271
1,10,4,-0.955627
2,10,3,-1.994644
3,10,1,-3.932004
4,10,6,1.087403
5,10,2,-2.956383
6,10,3,-1.977250
7,10,5,0.024828
8,10,4,-0.938406
9,10,2,-2.923549


Add in the effect of this information by using it to calculate $p$, the probability of getting heads. Use the logistic function to perform the calculation. 

$p_{calc} = \frac{e^{(\beta_0 + \beta_1 x)}}{1 + e^{(\beta_0 + \beta_1 x)}}$

In [20]:
def binom_obs(params):
    beta_0, beta_1 = params
    p_calc = np.exp(beta_0 + beta_1 * data['mg_diff'])/(1 + np.exp(beta_0 + beta_1 * data['mg_diff']))
    prob_k_heads = comb(data['flips'], data['heads']) * p_calc**data['heads'] * (1 - p_calc)**(data['flips'] - data['heads'])
    return prob_k_heads

In [21]:
def MLE_binom_obs(params):
    return -np.sum(np.log(binom_obs(params)))

In [22]:
starting_vals = [0, 0]
bounds = [(-500, 500), (-500, 500)]
mle_model_obs = minimize(MLE_binom_obs, starting_vals, method='Nelder-mead', bounds=bounds)
mle_model_obs

 final_simplex: (array([[0.01219842, 0.4722356 ],
       [0.01214065, 0.47216246],
       [0.01229047, 0.47221689]]), array([26.34011598, 26.34011606, 26.34011611]))
           fun: 26.340115975911182
       message: 'Optimization terminated successfully.'
          nfev: 112
           nit: 58
        status: 0
       success: True
             x: array([0.01219842, 0.4722356 ])

The second coefficient, $\beta_1$, is positive, which suggests that the heavier the tails side is relative to the other side, the higher the chance of getting heads. 

Show the best value of $p_{calc}$ to demonstrate this.

In [23]:
beta_0, beta_1 = mle_model_obs.x
data['p_calc'] = np.exp(beta_0 + beta_1 * data['mg_diff'])/(1 + np.exp(beta_0 + beta_1 * data['mg_diff']))
data

,flips,heads,mg_diff,p_calc
0,10,4,-0.903271,0.397867
1,10,4,-0.955627,0.391960
2,10,3,-1.994644,0.282977
3,10,1,-3.932004,0.136504
4,10,6,1.087403,0.628482
5,10,2,-2.956383,0.200382
6,10,3,-1.977250,0.284646
7,10,5,0.024828,0.505980
8,10,4,-0.938406,0.393899
9,10,2,-2.923549,0.202878


The weight difference does seem to affect the probability of getting heads as hypothesized.